In [1]:
import requests
import requests_cache
from bs4 import BeautifulSoup
import time
import json
import re
import pandas as pd # Dùng để đọc và hiển thị file csv/tsv
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from requests_html import HTML

In [ ]:
def filter_by_response(response) -> bool:
    return ('\"incomplete_results\":true' not in response.text)
requests_cache.install_cache(expire_after=None, filter_fn=filter_by_response)

In [ ]:
url = "https://soundcloud.com/discover/sets/charts-top:all-music:vn"
outputFile = "/Users/lanmac/Study/output.tsv"

In [ ]:
browser = webdriver.Chrome()
browser.get(url)
browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [ ]:
html = browser.page_source

In [ ]:
soup = BeautifulSoup(html)

In [ ]:
posts = soup.find_all("li", "systemPlaylistTrackList__item")

for post in posts:
    user = post.find("a", "trackItem__username").get_text()
    print(user)
    
    track = post.find("a", "trackItem__trackTitle").get_text()
    print(track)
print(len(posts))

________________________________________

**Lấy tất cả tracks của một user**

In [7]:
url = "https://soundcloud.com/discover"

In [8]:
browser = webdriver.Chrome()
browser.get(url)

In [9]:
html = browser.page_source
soup = BeautifulSoup(html)

In [10]:
def get_all_tracks():
    # Đồng ý cookies
    cookies = browser.find_element(By.XPATH,"//button[@id='onetrust-accept-btn-handler']")
    cookies.click()
    
    # Chọn một tab genre (demo)
    time.sleep(2)
    genre = browser.find_element(By.XPATH,"//a[@class='playableTile__heading playableTile__mainHeading sc-truncate sc-type-light sc-text-secondary sc-font-light sc-link-dark sc-link-primary sc-text-h4']")
    genre.click()

    time.sleep(2)
    html = browser.page_source
    soup = BeautifulSoup(html)

    time.sleep(2)
    profile = browser.find_element(By.XPATH,"//a[@class='trackItem__username sc-link-light sc-link-secondary']")
    profile.click()
    
    
    #time.sleep(3)
    lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
    lastCount = 0
    match=False
    while(match==False):
        time.sleep(5)
        lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
        if lastCount==lenOfPage:
            match=True
        else: 
            lastCount = lenOfPage
            
    time.sleep(3)
    html = browser.page_source
    soup = BeautifulSoup(html)
    
    time.sleep(3)
    songs = soup.find_all("li", "soundList__item")
    for track in songs:
        title = track.find("a", "soundTitle__title").get_text()
        print(title)
    print(len(songs))

In [11]:
get_all_tracks()


Tell Ur Mom II - Winno (ft Heily)


Em 20 - Winno


'tim em ko kó một đường nào' - Winno (ft Heily)


Fancy - Winno (prod.aloha)


Hai Tám Bốn - Winno


K-TLE & CHLLDISH - Redd Ocean


mua` xuan^ ma^t' em anh di bo^c' da^u` - Green x Ccmk


How u feel ! K-TLE ft. Young Bo5 (Prod. Chlldish)


Bởi vì ai rồi cũng phải lớn khôn (CUZ)  _H$ Mẫn (prod.@soulju)


"Dongchenbat".flp


𝕮𝖔𝖑𝖔𝖗 𝖁𝖎𝖇𝖊𝖘 01 * Khong Gian Cau - Dewie


Lil Thang - pu ekvw Ft Hashone


Srry to my ladies + Young Bo5 prod. Icevxz


𝕮𝖔𝖑𝖔𝖗 𝖁𝖎𝖇𝖊𝖘  02 * Heart Medication - Dewie


XANGUTNGAN


Dear Teacher - Robe


Body High - SMVL X Winno


Dạo bước cùng rồng vàng - Winno (SunXiX flip)


MONALISA  -  $ICE ft DOBUI


Fin - track 2: Khonghoitiec (No regret) [21DAMIXTAPE] (prod by. Origami)


Fancy - Winno (slowed + reverb)


Timemkoke1linenao * winno rmx *


Bốp 2 - Winno (Godthic dissin)


'tim em ko kó một đường nào' - Winno, Heily (slowed + reverb)


YungP - RobLucci (prod. $tarr)


K-TLE *DERMVE THIEUBONGEM* Ft Winno Prod. Ch